# $\textbf{Value function approximation}$

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from numba import njit

import Value_fun_NI as res0
import I0_no_func as res1
import parameters as params

import torch

In [2]:
# Time span for integration#parameters

T = params.T  # Total simulation time
d = params.d  # Number of states
psi = params.psi  # Transition rate matrix
n_intgr = params.n_intgr  # Number of intervals for numerical integration
n= params.n 

t_span = params.t_span

λ_init, α, β, η, xi = params.λ_init, params.α, params.β, params.η, params.xi

pi0 = params.pi0 
kappa = params.kappa

beta = β
lambda_infty = params.lambda_infty

rho = params.rho
m1 = params.m1
c = params.c
e = params.e
eta = params.eta
c_prime = params.c_prime
c0 = params.c0
nu_0 = params.nu_0
t_span = (0, 10)  # Integration time from 0 to 10

fig = plt.figure(figsize=(9,9))

<Figure size 900x900 with 0 Axes>

In [3]:
# Load the state_dict
def model_weights(file_path = 'i1net_model_sigmoid_switch.pth'):
    state_dict = torch.load(file_path, map_location=torch.device('cpu'))

    # Extract the weights and biases and convert them to lists
    weights_1 = state_dict['fc1.weight'].cpu().tolist()
    biases_1 = state_dict['fc1.bias'].cpu().tolist()
    weights_2 = state_dict['fc2.weight'].cpu().tolist()
    biases_2 = state_dict['fc2.bias'].cpu().tolist()
    weights_3 = state_dict['fc3.weight'].cpu().tolist()
    biases_3 = state_dict['fc3.bias'].cpu().tolist()
    weights_4 = state_dict['fc4.weight'].cpu().tolist()
    biases_4 = state_dict['fc4.bias'].cpu().tolist()

    return np.array(weights_1), np.array(biases_1), np.array(weights_2), np.array(biases_2), np.array(weights_3), np.array(biases_3), np.array(weights_4), np.array(biases_4)


In [4]:
# Define the sigmoid activation function
@njit
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Numba-compatible forward pass for I1_func_Net
@njit
def neural_net_forward_I1(t, u, y, weights):
    weights_1, biases_1, weights_2, biases_2, weights_3, biases_3, weights_4, biases_4 = weights
    x, d, kappa, mu = y
    kappa_flat = kappa.flatten()
    mu_flat = mu.flatten()
    
    # Creating the input vector
    inputs = np.array([t] + [u] + [x] + [d] + list(kappa_flat) + list(mu_flat), dtype=np.float64)
    
    # First layer
    x0 = np.dot(weights_1, inputs) + biases_1
    x0 = sigmoid(x0)
    
    # Second layer
    x0 = np.dot(weights_2, x0) + biases_2
    x0 = sigmoid(x0)
    
    # Third layer
    x0 = np.dot(weights_3, x0) + biases_3
    x0 = sigmoid(x0)
    
    # Output layer
    output = np.dot(weights_4, x0) + biases_4
    return output[0]

In [5]:
@njit
def W1(t, y_0, weights):
    y = (y_0[0], y_0[1], y_0[2][0], np.array(y_0[3][0]))
    # Generate grid points
    grid_points_all = np.linspace(0, T, 20)
    idx = np.searchsorted(grid_points_all, t)

    # Get the optimized result
    grid_points = grid_points_all[idx:]

    # Evaluate the function at grid points
    values = np.array([neural_net_forward_I1(t, x, y, weights) for x in grid_points])

    # Find the minimum value
    best_x_0 = np.max(values)
    return best_x_0

In [6]:
launch_train = True
load_model = False

N = 30

In [7]:
'''model_name = 'i1net_model_sigmoid_switch_second_train.pth'
weights = model_weights(model_name)
@njit
def W_previous(t, y_0):
    return W1(t, y_0, weights)'''

"model_name = 'i1net_model_sigmoid_switch_second_train.pth'\nweights = model_weights(model_name)\n@njit\ndef W_previous(t, y_0):\n    return W1(t, y_0, weights)"

In [ ]:
W_previous = res0.W0
model_name = 'i1net_model_sigmoid_switch_fourth_train.pth'

if launch_train == True:
    for i in range(N):
        model, losses = res1.Train_I1_func(W_previous, 200000, 200, load_model, model_name)
        load_model = True
        print('N: {}, Loss: {}'.format(i+1, losses[-1]))

        model_name = 'i{}net_model_sigmoid_switch_fifth_train_latest.pth'.format(i+1)
        torch.save(model.state_dict(), model_name)

        weights = model_weights(model_name)
        
        @njit
        def W_previous(t, y_0):
            return W1(t, y_0, weights)

/Users/yadhhafsi/PycharmProjects/OE_under_II/I0_no_func.py:180: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision training
/Users/yadhhafsi/PycharmProjects/OE_under_II/.venv/lib/python3.13/site-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(
  0%|          | 330/200000 [00:06<20:13, 164.59it/s] 